# MBC 뉴스

## Daily

In [66]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter


HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


DRIVER_PATH = '/Users/kimminho/chromedriver'
keywords = ['초록우산어린이재단', '산업혁명', '이해찬', '강원도', '신한은행', '상견례', '춘천시장', '카카오커머스', '초록우산', '오징어게임', 'sk가스', '블랙핑크', '4차산업혁명위원회', '오대훈', '디카르고', '행복', '최문순', '원주시', '맨유', 'mz세대', '아이폰13', '아스트라', '야구', '도이치모터스', '한국전력', '국세청', '20대', '오징어', '출마', '무소속', '위하고', '월드시리즈', '영림원', 'EY한영', '할인', '얀센', '대선', '티맥스', '나경원', '카카오커머스병합', '소나기', '빅뱅', '국민의힘', '디피', '산업단지관리공단', '누누', '솔샤르', '안철수', '육군', 'minsriver', '한국전력공사', '정세균', '이재수', '중앙지검', '오세훈', '더존비즈온', '백신', '모더나', '이낙연', '마스크', '로지스팟', '녹십자홀딩스', '매출채권팩토링', '광주', '이명박', '비트코인', '강원대학교', '이벤트', '개리', '탬파베이', '얀센백신', '지원금', 'mlb', '홍준표', '맨체스터유나이티드', '콘테', '황상무', '무지개', '삼성라이온즈', '화천대유', '정책', '게임', '유승민', '삼일회계법인', '이광재', '제주도', '키워드', '권지용', '여행', '테스트', '강릉', '윤희숙', '화이자백신', '문재인', '바베큐', '더존', 'GC녹십자홀딩스', '창원시', '민주당', '강민', '배구', '이재명', '코로나', '춘천', '윤석열', '4차위', '부산', '웹케시', '송영길', '애틀란타브레이브스', '노트북', '메타버스', '겨울', '축구', '공약', '김인규', '리버풀', '현대백화점', '김연경', '사천', '월드컵', '카카오', '한림대학교', '날씨', '공정', '정윤환', '야구장', '김용택', 'NFT', '트위터', '화이자', '향수', 'ERP']
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

for keyword in keywords:
    time.sleep(2)
    raw_target_url = f"https://imnews.imbc.com/more/search/?search_kwd={keyword}#page=0"
    driver.get(raw_target_url)
    time.sleep(1)
    driver.find_element_by_css_selector(
    '#result > div.section.content_spot > div.search_area.search_main.wrapper > form > fieldset > div.box_button > button.btn_sbar.btn_option').click()
    driver.find_element_by_css_selector(
    '#result > div.section.content_spot > div.search_area.search_option.wrapper.on > form > fieldset > div.option_list.option_date > div > div.date_select > a').click()
    driver.find_element_by_css_selector(
    '#result > div.section.content_spot > div.search_area.search_option.wrapper.on > form > fieldset > div.option_list.option_date > div > div.date_select > ul > li:nth-child(2) > a').click()
    driver.find_element_by_css_selector(
    '#result > div.section.content_spot > div.search_area.search_option.wrapper.on > form > fieldset > div.option_list.option_date > div > div.date_term > button').click()
    time.sleep(1)

    raw_html = driver.page_source
    raw_soup = BeautifulSoup(raw_html, 'lxml')

    print(raw_soup.find('span', {'class':'result_none'})['style'])
    if raw_soup.find('span', {'class':'result_none'})['style'] == "display: inline;":
        print("검색결과 없음")
        continue
        
#     총 게시글 수
    post_cnt = int(raw_soup.find('strong', {'class': 'num'}).get_text().replace(',', '').strip())

    total_page_cnt = (post_cnt // 5) + 2
    print(total_page_cnt)
    
    
    for i in range(0, total_page_cnt):

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        news_post_list = soup.find_all('li', {'class': 'item'})

        for news_post in news_post_list:
            thumbUrl = "https:" + news_post.find('img')['src']
            url = 'https://imnews.imbc.com' + news_post.find('a')['href']
            title = news_post.find('span', {'class': 'tit ellipsis2'}).get_text().strip()
            body = news_post.find('span', {'class': 'description ellipsis2'}).get_text().strip()
            category = news_post.find('span', {'class': 'category'}).get_text().strip()
            reporter = news_post.find('span', {'class': 'reporter'}).get_text().strip()

            post_date = news_post.find('span', {'class': 'date'}).get_text().strip()
            tmp_date = time.mktime(datetime.datetime.strptime(post_date, '%Y-%m-%d').timetuple())
            date = datetime.datetime.fromtimestamp(tmp_date)
            dt_hour = date.strftime('%Y.%m.%d %H')

            temp_id = str(url)
            temp_id = temp_id.encode('utf-8')
            h = hashlib.new('md5')
            h.update(temp_id)
            _id = h.hexdigest()

            source_emo = None
            keyword_rk = {}
            keyword_ak = {}
            collected_date = datetime.datetime.now().isoformat()

            data = {
                '_id': _id,
                'title': title,
                'body': body,
                'category': category,
                'date': date,
                'dt_hour': dt_hour,
                'reporter': reporter,
                'url': url,
                "count_ak": keyword_ak,
                "count_rk": keyword_rk,
                "emotions": source_emo,
                'collected_date': collected_date
            }
            print(data)
        # 다음 버튼 클릭
        try:
            driver.find_element_by_css_selector(
                "#result > div.section.content_area > div > div.result_list.wrapper > div > div > a.page_arr.next").click()
            time.sleep(0.5)
        except Exception as e:
            print("페이지 끝")
            continue


display: inline;
검색결과 없음
display: inline;
검색결과 없음
display: inline;
검색결과 없음
display: none;
3
{'_id': 'a2895ca072b205afca9debdbecfb16e7', 'title': '[스트레이트] 1조 원대 화물차 번호판 암시장 추적', 'body': "오늘 저녁 8시 20분에 방송된 MBC 탐사기획 가 1조 원대로 추정하는 화물차 번호판 시장에서 벌어지고 있는 부당거래를 추적했다. 현재 전국을 누비고 있는 화물차 50만대 중 약 10%인 5만 대가 '가짜 번호판'을 달고 있다는 충격적인 내용이었다. 시청에 등록도 하고 세금도 냈는데 내 번호판이 가짜? 는 '가짜 번호판' 피해를 당한 화물차 운전자들과 운송회사 관계자들을 만났다. 부산과 광주를 오가며 컨테이너 화물을 운송하고 있는 30대 김 모 씨는 얼마 전 양산시청에서 '번호판이 가짜이기 때문에 운행을 정지시키겠다'는 경고장을 받았다. 번호판이 컨테이너 화물차용 번호판이 아니라는 것이었다. 그러면서 정부의 유류세 지원도 끊겼다. 황당한 건 김 씨가 양산시에 정상적으로 차량을 등록하고 세금도 꼬박꼬박 내왔다는 점이었다. 김 씨에게 번호판을 대여한 운송회사도 이런 사실을 전혀 몰랐다. 전남 담양의 운송회사에서 이 번호판을 사왔는데 마찬가지로 문제없는 번호판으로 알고 사왔다고 했다. 차량의 주민등록 등초본이라고 할 수 있는 자동차 등록원부에도 아무 이상이 없었다. 는 한 지자체의 협조를 얻어 이 번호판의 전산 기록을 거슬러 올라갔다. 그랬더니 원래 번호판이 달려있던 차량이 견인차라는 것을 발견했다. 보통 시세가 1천만 원 정도인 견인차용 번호판이 누군가의 손을 거쳐 시세가 5천만 원이 넘는 컨테이너 화물차용 번호판으로 바뀐 것이었다. 는 서울 영등포, 인천 강화, 경북 포항 등에서도 비슷한 피해가 반복되고 있는 것을 확인했다. 오랜 기간 화물차 운송업계에서 일한 제보자는 2004년 영업용 화물차 번호판 발급이 허가제로 바뀌면서 규제가 

{'_id': '7aa34e2384c813e317220a276d40c161', 'title': '전국 소방서에 놓여진 요소수들', 'body': '훈훈한 소식도 이어지고 있습니다. 오늘 오전 경남 김해 서부소방서의 한 119안전센터에 한 남성이 이렇게 아무 말 없이 요소수를 놓고 사라졌는데요. 이곳뿐만 아니라 인천 송도, 강원도 춘천, 전북 전주 등 전국의 119 안전센터에 요소수를 놓고 가는 시민들의 발길이 이어지고 있습니다. 소방당국은 이들이 요소수가 떨어져서 119 출동이 늦어지는 걸 걱정해 요소수를 기부한 것으로 보고 있습니다.', 'category': '뉴스데스크', 'date': datetime.datetime(2021, 11, 7, 0, 0), 'dt_hour': '2021.11.07 00', 'reporter': '뉴스데스크 편집팀', 'url': 'https://imnews.imbc.com/replay/2021/nwdesk/article/6312891_34936.html', 'count_ak': {}, 'count_rk': {}, 'emotions': None, 'collected_date': '2021-11-08T16:43:44.039017'}
{'_id': '7c27a207b9efe4ef628af14ee11b7df7', 'title': '전국 곳곳 소방서 입구에 요소수 두고가는 시민들 잇따라', 'body': '전국에 요소수 품귀 현상이 빚어지는 가운데 119안전센터 앞에 요소수를 두고 사라지는 시민들의 발길이 이어지고 있습니다. 경남소방본부에 따르면 오늘 오전 8시에서 9시 사이 김해서부소방서 율하, 장유, 진례119안전센터 입구에 10ℓ 요소수 박스가 잇따라 놓여졌습니다. 한 남성이 율하에 3통, 장유에 1통, 진례에 1통씩 요소수를 두고 사라졌고, 또 다른 남성도 비슷한 시간대에 장유에 요소수 3통을 기부하고 자취를 감췄습니다. 소방 당국은 이들이 요소수 품귀 현상으로 119 출동이 지연되는 상황을 우려해 요소수를 기부한 것으로 보고 

KeyError: 'style'

# SBS 뉴스

In [59]:
for day in range(10):
    today = datetime.datetime.now()

    start_date = (today - datetime.timedelta(day)).strftime('%Y.%m.%d')
    end_date = (today - datetime.timedelta(day)).strftime('%Y.%m.%d')
    print(start_date, end_date)

2021.11.09 2021.11.09
2021.11.08 2021.11.08
2021.11.07 2021.11.07
2021.11.06 2021.11.06
2021.11.05 2021.11.05
2021.11.04 2021.11.04
2021.11.03 2021.11.03
2021.11.02 2021.11.02
2021.11.01 2021.11.01
2021.10.31 2021.10.31


In [64]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter


HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


DRIVER_PATH = '/Users/kimminho/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

keywords = "ㅁㄴㅇㅁㅇ윤석열"
today = datetime.datetime.now()
start_date = today.strftime('%Y.%m.%d')
end_date = today.strftime('%Y.%m.%d')

for keyword in keywords:
    raw_sbs_url = f'https://news.sbs.co.kr/news/search/main.do?pageIdx=1&searchTermStartDate={start_date}&searchTermEndDate={end_date}&query={keyword}'

    driver.get(raw_sbs_url)
    time.sleep(3)

    raw_html = driver.page_source
    raw_soup = BeautifulSoup(raw_html, 'lxml')

    if raw_soup.find('ul', {'class': 'psearch_result_list'}) is None:
        print("게시글이 없습니다.")
        continue
        
    temp_total_cnt = raw_soup.find('strong', {'class':'psr_num'}).get_text()
    total_cnt = int(temp_total_cnt.split('/')[1][:-1])

    print(total_cnt)
    if total_cnt == 0:
        print(raw_sbs_url)
        print("검색결과가 없습니다.")
        continue

    pageNo = round(total_cnt/10)+1

    for pageno in range(1, pageNo):
        sbs_url = f'https://news.sbs.co.kr/news/search/main.do?pageIdx={pageno}&searchTermStartDate={startDate}&searchTermEndDate={endDate}&query={keyword}'
        driver.get(sbs_url)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        news_post_list = soup.find('ul', {'class': 'psearch_result_list'}).find_all('li')

        for news_post in news_post_list:
            url = news_post.find('a',{'class':'psil_link'})['href']
            post_title = news_post.find('strong',{'class':'psil_tit'}).get_text()
            body = news_post.find('p',{'class':'psil_txt'}).get_text().replace('\t','').replace('\n','')
            temp_writer = news_post.find('span',{'class':'psil_info'}).get_text().replace('\t','').replace('\n','').replace(' ','').split('|')
            news = temp_writer[0]
            writer = temp_writer[1]
            post_date = temp_writer[2]
            category = temp_writer[3]

            tmp_date = time.mktime(datetime.datetime.strptime(post_date, '%Y.%m.%d').timetuple())
            date = datetime.datetime.fromtimestamp(tmp_date)
            dt_hour = date.strftime('%Y.%m.%d %H')

            temp_id = str(url)
            temp_id = temp_id.encode('utf-8')
            h = hashlib.new('md5')
            h.update(temp_id)
            _id = h.hexdigest()

            source_emo = None
            keyword_rk = {}
            keyword_ak = {}
            collected_date = datetime.datetime.now().isoformat()

            data = {
                "_id": _id,
                "post_title" : post_title,
                "writer" : writer,
                "body": body,
                "news": news,
                "date": date,
                "dt_hour":dt_hour,
                "category" : category,
                "url":url,
                "count_rk":keyword_rk,
                "count_ak":keyword_ak,
                "emotions":source_emo,
                "collected_date":collected_date
            }
            print(data)
            print('-') 

게시글이 없습니다.
0
https://news.sbs.co.kr/news/search/main.do?pageIdx=1&searchTermStartDate=2021.11.09&searchTermEndDate=2021.11.09&query=ㅁ
검색결과가 없습니다.
게시글이 없습니다.
0
https://news.sbs.co.kr/news/search/main.do?pageIdx=1&searchTermStartDate=2021.11.09&searchTermEndDate=2021.11.09&query=ㄴ
검색결과가 없습니다.


KeyboardInterrupt: 

In [5]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter


HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


DRIVER_PATH = '/Users/kimminho/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
keyword = '방탄소년단'
raw_sbs_url = f'https://news.sbs.co.kr/news/search/main.do?pageIdx=1&searchTermStartDate=&searchTermEndDate=&query={keyword}'

driver.get(raw_sbs_url)
time.sleep(1)

raw_html = driver.page_source
raw_soup = BeautifulSoup(raw_html, 'lxml')

if raw_soup.find('ul', {'class': 'psearch_result_list'}) is None:
    print("게시글이 없습니다.")


temp_total_cnt = raw_soup.find('strong', {'class': 'psr_num'}).get_text()
total_cnt = int(temp_total_cnt.split('/')[1][:-1].replace(',',''))

pageNo = round(total_cnt / 10) + 2

for pageno in range(1, pageNo):
    sbs_url = f'https://news.sbs.co.kr/news/search/main.do?pageIdx={pageno}&searchTermStartDate=&searchTermEndDate=&query={keyword}'
    driver.get(sbs_url)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    news_post_list = soup.find('ul', {'class': 'psearch_result_list'}).find_all('li')

    for news_post in news_post_list:
        url = news_post.find('a', {'class': 'psil_link'})['href']
        post_title = news_post.find('strong', {'class': 'psil_tit'}).get_text()
        body = news_post.find('p', {'class': 'psil_txt'}).get_text().replace('\t', '').replace('\n', '')
        temp_writer = news_post.find('span', {'class': 'psil_info'}).get_text().replace('\t', '').replace(
            '\n', '').replace(' ', '').split('|')
        news = temp_writer[0]
        writer = temp_writer[1]
        post_date = temp_writer[2]
        try:
            category = temp_writer[3]
        except:
            category = "-"
        print(post_date)
        tmp_date = time.mktime(datetime.datetime.strptime(post_date, '%Y.%m.%d').timetuple())
        date = datetime.datetime.fromtimestamp(tmp_date)
        dt_hour = date.strftime('%Y.%m.%d %H')
        


2021.11.25
2021.11.25
2021.11.25
2021.11.25
2021.11.25
2021.11.24
2021.11.24
2021.11.24
2021.11.24
2021.11.24
2021.11.24
2021.11.24
2021.11.23
2021.11.23
2021.11.23
2021.11.23
2021.11.22
2021.11.22
2021.11.22
2021.11.22
2021.11.22
2021.11.22
2021.11.22
2021.11.22
2021.11.21
2021.11.20
2021.11.18
2021.11.17
2021.11.16
2021.11.16
2021.11.16
2021.11.12
2021.11.10
2021.11.07
2021.11.06
2021.11.04
2021.11.02
2021.11.01
2021.10.29
2021.10.29
2021.10.29
2021.10.29
2021.10.29
2021.10.29
2021.10.28
2021.10.27
2021.10.27
2021.10.26
2021.10.26
2021.10.25
2021.10.25
2021.10.25
2021.10.25
2021.10.25
2021.10.24
2021.10.24
2021.10.23
2021.10.22
2021.10.20
2021.10.19
2021.10.19
2021.10.15
2021.10.15
2021.10.15
2021.10.13
2021.10.09
2021.10.07
2021.10.07
2021.10.06
2021.10.06
2021.10.05
2021.10.04
2021.10.04
2021.10.01
2021.10.01
2021.09.30
2021.09.30
2021.09.28
2021.09.28
2021.09.28
2021.09.28
2021.09.27
2021.09.27
2021.09.27
2021.09.27
2021.09.26
2021.09.26
2021.09.24
2021.09.24
2021.09.23
2021.09.23

ValueError: time data '방송/연예' does not match format '%Y.%m.%d'

# KBS 뉴스

In [7]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import datetime
from datetime import timedelta
import random
import hashlib
from collections import Counter

def recent_one_day_click():
    driver.find_element_by_css_selector("#btn-search-opt").click()
    time.sleep(0.5)
    # 최근 1일 클릭
    driver.find_element_by_css_selector("#content > div > div.search-result > div.component > div > dl.select.col1 > dd > ul > li:nth-child(2) > label").click()
    time.sleep(0.5)
    
    
HEADERS_LIST = """Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/11.0.696.50
Mozilla/5.0 Slackware/13.37 (X11; U; Linux x86_64; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/12.0.742.91""".split('\n')
HEADERS_LIST = [x.strip() for x in HEADERS_LIST]
agent = random.choice(HEADERS_LIST)
headers = {"User-Agent": "{}".format(agent)}


DRIVER_PATH = '/Users/kimminho/chromedriver'
keywords = ['이해찬', "요소수asdqw", '초록우산어린이재단', '산업혁명' , '강원도', '신한은행', '상견례', '춘천시장', '카카오커머스', '초록우산', '오징어게임', 'sk가스', '블랙핑크', '4차산업혁명위원회', '오대훈', '디카르고', '행복', '최문순', '원주시', '맨유', 'mz세대', '아이폰13', '아스트라', '야구', '도이치모터스', '한국전력', '국세청', '20대', '오징어', '출마', '무소속', '위하고', '월드시리즈', '영림원', 'EY한영', '할인', '얀센', '대선', '티맥스', '나경원', '카카오커머스병합', '소나기', '빅뱅', '국민의힘', '디피', '산업단지관리공단', '누누', '솔샤르', '안철수', '육군', 'minsriver', '한국전력공사', '정세균', '이재수', '중앙지검', '오세훈', '더존비즈온', '백신', '모더나', '이낙연', '마스크', '로지스팟', '녹십자홀딩스', '매출채권팩토링', '광주', '이명박', '비트코인', '강원대학교', '이벤트', '개리', '탬파베이', '얀센백신', '지원금', 'mlb', '홍준표', '맨체스터유나이티드', '콘테', '황상무', '무지개', '삼성라이온즈', '화천대유', '정책', '게임', '유승민', '삼일회계법인', '이광재', '제주도', '키워드', '권지용', '여행', '테스트', '강릉', '윤희숙', '화이자백신', '문재인', '바베큐', '더존', 'GC녹십자홀딩스', '창원시', '민주당', '강민', '배구', '이재명', '코로나', '춘천', '윤석열', '4차위', '부산', '웹케시', '송영길', '애틀란타브레이브스', '노트북', '메타버스', '겨울', '축구', '공약', '김인규', '리버풀', '현대백화점', '김연경', '사천', '월드컵', '카카오', '한림대학교', '날씨', '공정', '정윤환', '야구장', '김용택', 'NFT', '트위터', '화이자', '향수', 'ERP']
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

for keyword in keywords:
    pageCnt = 1
    
    raw_kbs_url = f'https://news.kbs.co.kr/search/search.do?query={keyword}#{pageCnt}'
    driver.get(raw_kbs_url)
    time.sleep(0.5)

    
    raw_html = driver.page_source
    raw_soup = BeautifulSoup(raw_html, 'lxml')
    
    # 검색결과 없습니다.
    if raw_soup.find('div', {'class':'paging type1'})['style'] == "display:none":
        print("검색결과 없음")
        continue
    
    
    while True:
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        
        
        news_list = soup.find('ul', {'id':'searchList'}).find_all('li')
        
        for news in news_list:
            url = news.find('a')['href']
            # 본문 로직
            driver.get(url)
            time.sleep(0.5)
            
            in_html = driver.page_source
            in_soup = BeautifulSoup(in_html, 'lxml')
            
            landing_box = in_soup.find('div', {'class':'landing-box'})
            title = landing_box.find('h5', {'class':'tit-s'}).get_text()
            post_date = landing_box.find('em', {'class':'date'}).get_text()[2:].replace(' ','')
            body = landing_box.find('div', {'id':'cont_newstext'}).get_text().replace('\n','').replace('\t','').replace('/','').replace('촬영기자','')
            
            try:
                reporter_tag = landing_box.find('ul', {'id':'ulReporterList'})
                reporter_info = reporter_tag.find('p', {'class':'name'}).find_all('span')
                writer = reporter_info[0].get_text()
                
                category = landing_box.find('a', {'class':'cate'}).get_text()

            except Exception as e:
                print(e)
                writer = "-"
                category = "-"

            
            tmp_date = time.mktime(datetime.datetime.strptime(post_date, '%Y.%m.%d(%H:%M)').timetuple())
            date = datetime.datetime.fromtimestamp(tmp_date)
            dt_hour = date.strftime('%Y.%m.%d %H')
            
            temp_id = str(title)
            temp_id = temp_id.encode('utf-8')
            h = hashlib.new('md5')
            h.update(temp_id)
            _id = h.hexdigest()

            source_emo = None
            keyword_rk = {}
            keyword_ak = {}
            collected_date = datetime.datetime.now().isoformat()

            data = {
                '_id':_id,
                'title':title,
                'body':body,
                'category': category,
                'date':date,
                'dt_hour':dt_hour,
                'reporter':writer,
                'url':url,
                'broadcast':"KBS",
                'keyword':keyword,
                "count_ak": keyword_ak,
                "count_rk": keyword_rk,
                "emotions": source_emo,
                'collected_date': collected_date
            }
            
            print(data)
            
        print('-')
        pageCnt = pageCnt + 1
        next_kbs_url = f'https://news.kbs.co.kr/search/search.do?query={keyword}#{pageCnt}'
        driver.get(next_kbs_url)
        time.sleep(0.5)
        
        if soup.find('div', {'class':'section result-none'})['style'] == "display:block":
            print("검색결과 없음")
            break
            
        
        if soup.find_all('a', {'class':'next disabled'}):
            print("더 이상 페이지가 없습니다.")
            break




{'_id': 'b768ce78fe8a9e57f4263b9f73fae141', 'title': '이해찬 “총선 때 검찰 정치공작 제보”…국민의힘 “정치공세 중단”', 'body': ' [앵커] 윤석열 전 총장을 둘러싼  \'고발 사주 의혹\'에 대해   정치권 파장이 계속되고 있습니다. 민주당 이해찬 전 대표는 지난해 총선 때 검찰의 공작이 있었다고 주장했고,   국민의힘은  흠집내기식 정치공세를  중단하라고 반박했습니다. 손서영 기자의 보도입니다. [리포트] 민주당에선   지난 총선 때 당을 이끌었던  이해찬 전 대표가 나섰습니다. 총선 직전, 검찰발 공작이 있다는 제보를 받았는데,   이번에 보니, 고발 사주 의혹이 그 공작인 것 같다고 밝혔습니다.[이해찬민주당 전 대표TBS 라디오 : "그때 저희한테 준 제보가 상당히 정확했다, 이렇게 생각이 들어요. 민주주의 체제를 말하자면 교란시키는 아주 국기 문란 행위라고 봐야죠."] 윤호중 원내대표는   정치 검사인 윤석열 전 총장의 실체가 드러나고 있다고 비판하고,   공수처를 향해  수사에 착수하라고 촉구했습니다. 민주당에선   윤석열 전 총장을 다음달 국정감사에 증인으로 채택하자는 주장도 나왔습니다. 윤석열 후보 측은  선거철만 되면 나타나는  판에 박힌 정치공작이라고 반박했습니다. 윤 전 총장에서부터   손준성 검사, 김웅 의원으로 고발 사주가 이어진다는 의혹도   전혀 실체가 없다고 주장했습니다. 국민의힘 김기현 원내대표도  단 한 톨도 문제될게  없다고 판단한다고 밝혔습니다.[김기현국민의힘 원내대표 : "고발사주란 용어자체가 틀렸고요. 공익제보 형태가 아닌가 저희들은 그렇게 판단하고 있습니다."] 다만 당 내부에서는  특정후보에 대한 의혹을  어디까지 방어해야 할지에 대한   고민도 있습니다. 고발장 전달자로 지목된 국민의힘 김웅 의원은 내일 기자 회견을 열고 입장을 밝힙니다. KBS 뉴스 손서영입니다. :최연송영상편집:이윤진', 'category': '뉴스7(제주)', 'date': datetime.d

{'_id': 'b768ce78fe8a9e57f4263b9f73fae141', 'title': '이해찬 “총선 때 검찰 정치공작 제보”…국민의힘 “정치공세 중단”', 'body': ' [앵커] 윤석열 전 총장을 둘러싼  \'고발 사주 의혹\'에 대해 정치권 파장이 계속되고 있습니다. 민주당 이해찬 전 대표는 지난해 총선 때 검찰의 공작이 있었다고 주장했고, 국민의힘은  흠집내기식 정치공세를  중단하라고 반박했습니다. 손서영 기자의 보도입니다. [리포트] 민주당에선 지난 총선 때 당을 이끌었던  이해찬 전 대표가 나섰습니다. 총선 직전, 검찰발 공작이 있다는 제보를 받았는데, 이번에 보니, 고발 사주 의혹이 그 공작인 것 같다고 밝혔습니다. [이해찬민주당 전 대표TBS 김어준의 뉴스공장 : "그때 저희한테 준 제보가 상당히 정확했다, 이렇게 생각이 들어요. 민주주의 체제를 말하자면 교란시키는 아주 국기 문란 행위라고 봐야죠."] 윤호중 원내대표는 정치 검사인 윤석열 전 총장의 실체가 드러나고 있다고 비판하고, 공수처를 향해  수사에 착수하라고 촉구했습니다. 민주당에선 윤석열 전 총장을 다음달 국정감사에 증인으로 채택하자는 주장도 나왔습니다. 윤석열 후보 측은  선거철만 되면 나타나는  판에 박힌 정치공작이라고 반박했습니다. 윤 전 총장에서부터 손준성 검사, 김웅 의원으로 고발 사주가 이어진다는 의혹도 전혀 실체가 없다고 주장했습니다. 국민의힘 김기현 원내대표도  단 한 톨도 문제될게  없다고 판단한다고 밝혔습니다. [김기현국민의힘 원내대표 : "고발사주란 용어자체가 틀렸고요. 공익제보 형태가 아닌가 저희들은 그렇게 판단하고 있습니다."] 다만 당 내부에서는  특정후보에 대한 의혹을  어디까지 방어해야 할지에 대한 고민도 있습니다. 고발장 전달자로 지목된 국민의힘 김웅 의원은 내일 기자 회견을 열고 입장을 밝힙니다. KBS 뉴스 손서영입니다. :최연송영상편집:이윤진', 'category': '뉴스7(청주)', 'date': datetime.datetime(2021

{'_id': 'a2b9a2d2d988c9a9f55f88cf95d9f927', 'title': '“민주당 ‘이해찬 장애인 비하발언’ 시정권고 받고도 일부만 이행”', 'body': '더불어민주당이 이해찬 전 대표의 장애인 비하 발언으로 국가인권위원회에서 시정 권고를 받았지만, 권고 내용 중 일부만 이행하겠다는 계획을 냈던 것으로 파악됐습니다. 전국장애인차별철폐연대에 따르면, 더불어민주당은 이 전 대표의 발언 자체에 대한 인권 교육이 아닌 ‘장애 인권 교육 계획’과 ‘재발 방지 대책’과 관련한 내용만 제출한 것으로 전해졌습니다. 이에 인권위는 민주당이 시정 권고를 ‘일부만 수용’했다고 판단했습니다. 반면 주호영 원내대표의 장애인 비하 발언으로 역시 인권위에서 같은 내용의 시정 권고를 받았던 국민의힘은, 인권위가 권고한 이행계획을 제출해 인권위에서 ‘전부 수용’ 결정을 받은 것으로 전해졌습니다. 앞서 전국장애인차별철폐연대 등은 민주당 이해찬 전 대표의 ‘선천적 장애인은 의지가 약하다’ 발언과 국민의힘 주호영 원내대표의 ‘절름발이 총리’ 발언에 대해 진정을 제기했습니다. 인권위는 두 정당에 재발방지 대책을 마련하고, 문제가 된 발언과 관련해 장애인 인권 교육을 하라고 권고했습니다. 인권위법에 따라 시정권고가 결정되면 권고 조치를 받은 피진정인은 90일 이내에 권고 조치 이행 계획을 인권위에 제출해야 합니다. 인권위는 이행 계획을 보고받은 뒤 이 내용이 인권위의 권고를 수용한 것인지, 아닌지를 결정합니다.[사진 출처 : 연합뉴스]', 'category': '사회', 'date': datetime.datetime(2021, 4, 19, 14, 33), 'dt_hour': '2021.04.19 14', 'reporter': '신지수 기자', 'url': 'http://news.kbs.co.kr/news/view.do?ncd=5165879', 'broadcast': 'KBS', 'keyword': '이해찬', 'count_ak': {}, 'count_rk': {}, 'em

{'_id': 'bbc0b92a42f5bb112ecabdf5b06b2aaa', 'title': '인권위, ‘이해찬 장애인 비하 발언’ 민주당에 재발 방지 권고', 'body': '국가인권위원회가 이해찬 더불어민주당 전 대표의 ‘장애인 비하 발언’과 관련해 민주당에 재발 방지 대책 마련 등을 권고한 것으로 파악됐습니다. 해당 권고는 사회 소수 집단에 대한 비하 표현을 진정 대상으로 삼고 권고 결정을 내린 첫 사례인 것으로 알려졌습니다. 인권위는 최근, “선천적 장애인은 의지가 약하다”라는 이 전 대표의 발언에 대한 권고 결정문을 민주당과 진정인 측에 보냈습니다. 앞서 인권위는 지난 8월, 전원위원회를 열어 민주당에 당직자가 장애인에 대한 비하 발언을 하지 않도록 재발 방지 대책을 마련하고 이 대표를 포함한 전체 당직자에게 장애인 인권교육을 실시할 것을 권고하는 결정을 내렸습니다. 이번 사건에서는 장애인차별금지 및 권리구제 등에 관한 법률(장애인차별금지법)이 근거가 됐습니다. 장애인차별금지법 제32조 3항은 누구든지 장애를 이유로 학교, 시설, 직장 지역사회 등에서 장애인 또는 장애인 관련자에게 모욕감을 주거나 비하를 유발하는 표현이나 행동을 하면 안 된다고 규정하고 있습니다.  인권위는 집단 자체를 모욕하거나 비하해 그 집단에 속하는 사람의 인격이 침해됐다고 볼 특별할 사정이 있으면 조사 대상으로 삼을 수 있다고 판단한 겁니다. 올해 초, 이 전 대표(당시 대표)는 민주당 공식 유튜브 채널에 출연해 “선천적 장애인은 어려서부터 장애를 가지고 나와서 의지가 약하다고 한다”라고 말해 논란이 됐습니다. 논란이 되자 이 전 대표는 “고의적으로 한 말은 아니고, 분석한 얘기를 들어서 전한 말인데, 결과적으로 여러 가지 상처를 줬다고 하면 죄송하다는 말씀을 드리겠다”고 사과했습니다.[사진 출처 : 연합뉴스]', 'category': '사회', 'date': datetime.datetime(2020, 11, 23, 8, 46), 'dt_hour': '2020.11.23 08

{'_id': '3db647ee4a5200d4a55cab5933348922', 'title': '32년 정치 일선 물러나는 이해찬 “남북관계 해결 아쉬워…지지율은 오르내리는 것”', 'body': '내일 민주당 전당대회를 끝으로 임기를 마치는 더불어민주당 이해찬 대표가 유튜브 생중계로 이뤄진 기자간담회에서 지난 소회를 밝혔습니다. 아쉬운 일로는 교착 상태에 빠진 남북 관계를, 잘한 일로는 올해 4.15 총선 성과를 꼽았습니다. 이 대표는 간담회에 앞선 인사말에서 "저는 내일(29일) 32년 간의 정치 생활을 마감하게 된다"며 국민과 당원에 감사의 뜻을 전했습니다. 이어진 기자간담회에서 이 대표는 \'당 대표로서 가장 잘 했다고 생각하는 점\'을 묻는 질문에 "2018년 전당대회에 출마하면서 당원 동지에게 약속했던 게 2020년 총선을 잘 준비해서 좋은 성과를 내겠다는 것"이었다며 국민들이 좋은 평가를 해줘 이번에 많은 의석을 얻었다고 말했습니다. 이른바 \'시스템 공천\'과 \'플랫폼 정당\'을 만든 일도 잘한 일이라고 이 대표는 자평했습니다. 아쉬운 일로는 남북 관계를 충분히 풀어나가지 못한 점을 꼽았습니다. 그러면서 이 대표는 "당분간 교착 상태라 하더라도 포기하지 않고 꾸준히 대화해나가는 건 결코 포기할 수 없는 길"이라며 공직 생활을 끝내더라도 동북아평화경제협회 이사장으로서 민간 교류 등에 힘쓰겠다고 설명했습니다. 최근 하락 추세를 보인 당 지지율에 대해서는 "일희일비하지 않는 게 좋다"고 말했습니다. 이 대표는 지지율 하락 원인이 무엇이라고 생각하느냐는 질문에 "실제로 어느 한 요인을 가지고 (지지율이) 크게 빠지거나 올라가는 게 아니다"라면서 "국민들에게 얼마나 진실하고 정성스럽게 임하느냐가 중요하다"고 답했습니다. 젊은 층 지지 이탈 원인으로 꼽힌 부동산 문제에 대해선 "부동산 정책을 주거 정책으로 봐야지, 투자 정책으로 접근하는 방식은 바람직하지 않다"며 "집이 없는 사람들이 안심하고 살 수 있는 주거 대책에 역점을 둬야 한다"고 강조했습니다. 

{'_id': 'd9ddc4144187810a4a67861863818ec5', 'title': '이해찬 “의사협회 파업 옳지 않아…대화로 문제 해결하자”', 'body': '더불어민주당 이해찬 대표가 "국민 모두 어려운 시간을 보내고 있는 때 의사들이 국민 건강과 생명을 담보로 파업을 하는 것은 옳지 않다"고 말했습니다. 이 대표는 오늘(14일) 당 최고위원회의에서 "정부가 협의체를 구성해 요구사항을 논의하자고 거듭 제안했는데도 (의사협회가) 이를 거부하고 집단행동에 나서는 것에 유감"이라며 이같이 밝혔습니다. 이 대표는 "지역 간 의료 불균형 완화와 공공의료 확대는 10년 전부터 제기된 문제이자 코로나 사태를 거치며 다시 확인된 과제"라며 "지금이라도 파업을 중단하고 우리나라 의료체계 개선을 위해 대화에 나설 것을 촉구한다"고 했습니다. 이어 "정부는 이번 파업에 원칙을 가지고 대응해달라"면서 "정부와 지자체는 이번 파업으로 국민 건강과 생명이 위협받지 않도록 만전을 기해달라"고 요청했습니다. 김태년 원내대표는 "의사협회는 집단 휴진을 철회하고 대화에 응해야 한다"면서 "보건당국과 의료단체는 동반자가 되어야 하는 만큼, 보건의료 현실에 대한 허심탄회한 대화를 통해 상생 발전하는 방안을 찾아야 한다"고 밝혔습니다.[사진 출처 : 연합뉴스]', 'category': '정치', 'date': datetime.datetime(2020, 8, 14, 10, 40), 'dt_hour': '2020.08.14 10', 'reporter': '조태흠 기자', 'url': 'http://news.kbs.co.kr/news/view.do?ncd=4517181', 'broadcast': 'KBS', 'keyword': '이해찬', 'count_ak': {}, 'count_rk': {}, 'emotions': None, 'collected_date': '2021-11-23T10:16:08.423434'}
{'_id': '63d92e983e010258ec9e6e5b06add117'

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=95.0.4638.69)
